In [3]:
import mysql.connector as MyConn
import os
import sys
from datetime import date, timedelta
from yahoo_fin.stock_info import get_data
import yfinance as yf
pwd = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(pwd)
from functions.list_of_data import get_tickers
from functions.insert_data_in_sql_yfinance import insert_yfinance_data
from functions.insert_data_in_sql_yfinance import insert_latest_yfinance_data_   # this is better we will chooose this 
tickers = get_tickers()
today = date.today()
from_day = today - timedelta(days=(365 * 28) + 1)
print('total_tickers -->',len(tickers))
tickers = tickers[0:35]

for index,ticker in enumerate(tickers,start=1):
    try:
        data = get_data(ticker, start_date=str(from_day), end_date=str(today))
        data = data.reset_index()
        data = data.rename(columns={'index':'date'})
        data_stored = insert_latest_yfinance_data_(ticker=ticker,data=data)
        print(f'{index} -->{data_stored}')
    except Exception as e:
        print(f'{ticker} --> not found, because of {e}')

total_tickers --> 6813
1 -->20MICRONS.NS data stored in SQL database
2 -->21STCENMGM.NS data stored in SQL database
3 -->360ONE.NS data stored in SQL database
4 -->3IINFOLTD.NS data stored in SQL database
5 -->3MINDIA.NS data stored in SQL database
6 -->3PLAND.NS data stored in SQL database
7 -->5PAISA.NS data stored in SQL database
8 -->63MOONS.NS data stored in SQL database
9 -->AMJUMBO.NS data stored in SQL database
10 -->ABB.NS data stored in SQL database
11 -->ABINFRA.NS data stored in SQL database
12 -->ABMINTLLTD.NS data stored in SQL database
13 -->ABREL.NS data stored in SQL database
14 -->AGUL.NS data stored in SQL database
15 -->A2ZINFRA.NS data stored in SQL database
16 -->AAATECH.NS data stored in SQL database
17 -->AADHARHFC.NS data stored in SQL database
18 -->AAKASH.NS data stored in SQL database
19 -->AAREYDRUGS.NS data stored in SQL database
20 -->AARON.NS data stored in SQL database
21 -->AARTECH.NS data stored in SQL database
22 -->AARTIDRUGS.NS data stored in SQL d

'20MICRONS.NS stored in SQL database'

In [11]:
# for database only
import pandas as pd
from sqlalchemy import create_engine
username = os.getenv('USERNAME_SQL')
password = os.getenv('PASSWORD_SQL')
host = os.getenv('HOST_SQL')
connection = MyConn.connect(
    host=host,
    user=username,
    password=password
)
if connection.is_connected():
    print("Successfully connected to the database")
    cursor = connection.cursor()

    # SQL query to create a new database
    create_database_query = "CREATE DATABASE IF NOT EXISTS stock_data_db;"
    cursor.execute(create_database_query)
    print("Database 'stock_data_db' created successfully or already exists!")

    # Selecting the newly created database
    cursor.execute("USE stock_data_db")

    # SQL query to check if the table exists
    check_table_query = f"""
    SELECT COUNT(*)
    FROM information_schema.tables
    WHERE table_schema = DATABASE() AND table_name = '{ticker}';
    """
    cursor.execute(check_table_query)
    table_exists = cursor.fetchone()[0]

    # If the table exists, truncate it; otherwise, create it
    if table_exists:
        truncate_table_query = f"TRUNCATE TABLE `{ticker}`;"
        cursor.execute(truncate_table_query)
        print(f"Table `{ticker}` truncated successfully.")
    else:
        # SQL query to create the table if it does not exist
        create_table_query = f"""
        CREATE TABLE `{ticker}` (
            date DATE NOT NULL,
            open FLOAT,
            high FLOAT,
            low FLOAT,
            close FLOAT,
            adjclose FLOAT,
            volume INT,
            ticker VARCHAR(50) NOT NULL,
            PRIMARY KEY (date, ticker)
        );
        """
        cursor.execute(create_table_query)
        print(f"Table `{ticker}` created successfully.")

    # Insert data into the table
    for index, row in data.iterrows():
        insert_query = f"""
            INSERT INTO `{ticker}` (date, open, high, low, close, adjclose, volume, ticker)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
        """
        values = (
            row['date'],  # Assume 'date' is already in a valid datetime format
            float(row['open']) if pd.notna(row['open']) else None,
            float(row['high']) if pd.notna(row['high']) else None,
            float(row['low']) if pd.notna(row['low']) else None,
            float(row['close']) if pd.notna(row['close']) else None,
            float(row['adjclose']) if pd.notna(row['adjclose']) else None,
            int(row['volume']) if pd.notna(row['volume']) else None,
            row['ticker'] if pd.notna(row['ticker']) else None
        )
        print(values)
        cursor.execute(insert_query, values)
    
    connection.commit()
    print("Data inserted successfully into the table!")

    # Closing the cursor and connection
    cursor.close()
    connection.close()

Successfully connected to the database
Database 'stock_data_db' created successfully or already exists!
Table `20MICRONS.NS` truncated successfully.
(Timestamp('2008-10-06 00:00:00'), 40.0, 40.0, 15.800000190734863, 16.774999618530273, 14.652863502502441, 23501600, '20MICRONS.NS')
(Timestamp('2008-10-07 00:00:00'), 16.0, 19.0, 13.925000190734863, 15.024999618530273, 13.124247550964355, 9113400, '20MICRONS.NS')
(Timestamp('2008-10-08 00:00:00'), 14.0, 14.600000381469727, 12.550000190734863, 13.25, 11.573797225952148, 2464384, '20MICRONS.NS')
(Timestamp('2008-10-10 00:00:00'), 12.449999809265137, 12.449999809265137, 10.824999809265137, 11.600000381469727, 10.13253116607666, 1207928, '20MICRONS.NS')
(Timestamp('2008-10-13 00:00:00'), 12.149999618530273, 13.300000190734863, 11.649999618530273, 12.324999809265137, 10.765814781188965, 898692, '20MICRONS.NS')
(Timestamp('2008-10-14 00:00:00'), 12.824999809265137, 13.0, 10.75, 11.074999809265137, 9.673946380615234, 673088, '20MICRONS.NS')
(Tim

In [5]:
data

,date,open,high,low,close,adjclose,volume,ticker
0,2008-10-06,40.000000,40.000000,15.800000,16.775000,14.652863,23501600.0,20MICRONS.NS
1,2008-10-07,16.000000,19.000000,13.925000,15.025000,13.124249,9113400.0,20MICRONS.NS
2,2008-10-08,14.000000,14.600000,12.550000,13.250000,11.573797,2464384.0,20MICRONS.NS
3,2008-10-10,12.450000,12.450000,10.825000,11.600000,10.132533,1207928.0,20MICRONS.NS
4,2008-10-13,12.150000,13.300000,11.650000,12.325000,10.765815,898692.0,20MICRONS.NS
...,...,...,...,...,...,...,...,...
3991,2024-12-09,261.929993,262.000000,248.830002,251.770004,251.770004,75340.0,20MICRONS.NS
3992,2024-12-10,250.979996,253.850006,244.000000,247.539993,247.539993,59651.0,20MICRONS.NS
3993,2024-12-11,249.000000,249.899994,239.000000,244.649994,244.649994,101148.0,20MICRONS.NS
3994,2024-12-12,244.699997,256.880005,241.000000,256.019989,256.019989,119116.0,20MICRONS.NS


In [ ]:
import mysql.connector as MyConn
import os
import sys
from datetime import date, timedelta

import yfinance as yf
pwd = os.path.abspath(os.path.join(os.path.dirname(__file__), ".."))
sys.path.append(pwd)
from functions.list_of_data import get_tickers
tickers = get_tickers()
today = date.today()
from_day = today - timedelta(days=(365 * 28) + 1)
tickers = tickers[0:10]

for dx,ticker in enumerate(tickers):
    try:
        data = yf.download(tickers=ticker,start=str(from_day),end = str(today),interval='1d')
        print(ticker)
        # data = get_data(ticker, start_date=str(from_day), end_date=str(today))
    except:
        print('ticker not found')
    break




# for database only
# username = os.getenv('USERNAME_SQL')
# password = os.getenv('PASSWORD_SQL')
# host = os.getenv('HOST_SQL')
# connection = MyConn.connect(
#     host=host,
#     user=username,
#     password=password
# )
# if connection.is_connected():
#     print("Successfully connected to the database")
#     cursor = connection.cursor()

#     # SQL query to create a new database
#     create_database_query = "CREATE DATABASE IF NOT EXISTS stock_data_db;"
#     cursor.execute(create_database_query)
#     print("Database 'stock_data_db' created successfully or already exists!")

#     # Selecting the newly created database
#     cursor.execute("USE stock_data_db")

#     # SQL query to create the table
#     create_table_query = """
#     CREATE TABLE IF NOT EXISTS stock_data (
#         date DATE,
#         ticker VARCHAR(10),
#         open FLOAT,
#         close FLOAT,
#         adj_close FLOAT,
#         high FLOAT,
#         low FLOAT
#     );
#     """
#     # Executing the create table query
#     cursor.execute(create_table_query)
#     print("Table 'stock_data' created successfully!")

#     # Closing the cursor and connection
#     cursor.close()
#     connection.close()